<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_Healthcare/blob/lesson_3_matching/GB_nlp_healthcare_3_Berezutskii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание задания

- **Описание задания**:
Когда пациент принимает какое-либо лекарство, у него может возникнуть побочный эффект от лекарства или побочную реакцию на лекарство (ADR). Мы можем использовать [CUI](https://www.nlm.nih.gov/research/umls/new_users/glossary.html#:~:text=CUI%20-%20The%20Concept%20Unique%20Identifier%20for%20a%20Metathesaurus%20concept%20to%20which%20strings%20with%20the%20same%20meaning%20are%20linked.%20One%20of%20the%20principles%20of%20the%20Metathesaurus%20is%20that%20meanings%20should%20be%20preserved%20over%20time%20regardless%20of%20what%20terms%20(atoms)%20are%20used%20to%20express%20those%20meanings.), уникальный идентификатор концепта для  Metathesaurus, чтобы определить, какие нежелательные реакции упоминаются в сообщениях пациентов. обзоры лекарств в структурированном виде.

   Цель этого задания – помочь вам научиться идентифицировать (ADR) по описаниям пациентов о нежелательных реакциях на лекарственные препараты. Вы сможете:
  - Используйте подход NLP для извлечения побочных эффектов из обзоров лекарств.




## Предоставленные файлы
Один из файлов, которые у вас есть для этого задания, называется «побочные_эффекеты.txt». Обзоры лекарств разделены предложением. Для каждой строки есть «row_id» и «sentence_text», разделенные табуляцией.

> Вот пример:
`7<tab>I was unable to sleep, had blurred vision, and felt sick to my stomach.`

Другой файл называется CUI_concepts.txt. АПИ и текст его концепции указаны в каждой строке.

> Вот пример:
`C0344232<tab>Blurred vision,Blurring of visual image`

In [ ]:
import pandas as pd

from string import punctuation as punct # пунктуация
from nltk.stem.porter import PorterStemmer # стеммер

from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

from nltk.corpus import stopwords
import gensim
import gensim.downloader as api
from gensim.models import FastText, Word2Vec, KeyedVectors

import torch
from transformers import AutoTokenizer, AutoModel

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Шаг 1: Чтение файла данных
- Попробуйте распечатать несколько строк и посмотреть примеры в наборе данных, с которым вы работаете. Вы будете протестированы на небольшом подмножестве из них, но ваши функции должны применяться в целом.

In [ ]:
with open('/content/побочные_эффекеты.txt') as file:
  documents = file.readlines()

# напечатаем первые 5 строк файла.
# Не стесняйтесь просматривать любое количество
print(*documents[:5], sep = '\n')

id	Text

1	extreme weight gain, short-term memory loss, hair loss.

2	COMPLETELY DESTROYED SEXUALLY FUNCTIONING .

3	Just TWO tablets of Lexapro 10mg completely destroyed my sexual functioning, probably for life.

4	It's called PSSD: post-SSRI sexual dysfunction.



# Шаг 2. Считайте файл CUI и сохраните его для удобного поиска.
- Сохраните все CUI и соответствующие им ключевые слова.
— Идея такая: пройтись по каждому CUI, чтобы определить, присутствует ли он в данном текстовом сегменте.

In [ ]:
cuinames = []
concepts = []
with open('/content/концепты.tsv') as cuifile: # CUI явления
  print(next(cuifile))  # пропустите заголовок, распечатайте, чтобы увидеть структуру строки
  for i, line in enumerate(cuifile):
    fields = line.split('\t')
    # удалите конечные пробелы (и нечетные случайные nbsp...)
    cui = fields[0].strip()
    # храните различные текстовые представления концептов
    # но отбросьте дублирующиеся текстовые представления для одного и того же концепта
    texts = set(f.strip() for f in fields[1].split(','))
    for t in texts:
      # всегда добавляйте вместе, чтобы обеспечить одинаковый индекс
      cuinames.append(cui)
      concepts.append(t.strip().lower())

CUI	CONCEPT	SNOMED_CODE



In [ ]:
print(len(concepts), len(cuinames))  # проверка на вменяемость
# распечатайте несколько CUIs. Исследуйте CUIs, пробуя различные диапазоны,
# или распечатайте весь список, если хотите.
for i in list(range(42,45)):
  print('CUI: {}\tConcept: {}'.format(cuinames[i], concepts[i]))

738 738
CUI: C0231528	Concept: myalgia
CUI: C0009676	Concept: confusion
CUI: C0009676	Concept: confusional state


# Создание датафреймов

Для обработки полученных данных в ходе извлечения концептов создадим несколько датафреймов

## Датафрейм с id предложения и текстом, который будем обрабатывать.

**В изначальном файле, примерно на 635 строке был перенос строки и точка, поэтому код ниже не сработает, если просто запустить. Этот перенос строки исправлял вручную.**

**Также в документе существовало предложение с id = 0**

In [ ]:
file_path = '/content/побочные_эффекеты.txt'

# Чтение файла и создание DataFrame
df = pd.read_csv(file_path, sep='\t', header=None, names=['sentense_id', 'text'])
df = df.drop(labels=0, axis=0)
# Вывод полученного DataFrame
df.head()

,sentense_id,text
1,1,"extreme weight gain, short-term memory loss, h..."
2,2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING .
3,3,Just TWO tablets of Lexapro 10mg completely de...
4,4,It's called PSSD: post-SSRI sexual dysfunction.
5,5,And there is a chance that it will give you PS...


In [ ]:
# выведем тестовую строку
first_sentence = df.iloc[0][1]
first_sentence

'extreme weight gain, short-term memory loss, hair loss.'

## Датафрейм с концептами и CUI

In [ ]:
df_concepts = pd.DataFrame()
df_concepts['CUI'] = cuinames
df_concepts['concepts'] = concepts
df_concepts.drop_duplicates(inplace=True)
df_concepts.head()

,CUI,concepts
0,C0000765,excessive weight gain
1,C0000765,excessive body weight gain
2,C0701811,poor short-term memory
3,C0002170,alopecia
4,C0002170,loss of hair


In [ ]:
df_concepts[df_concepts['CUI'] == 'C0002170']

,CUI,concepts
3,C0002170,alopecia
4,C0002170,loss of hair


# Шаг 3: Извлечение концептов



## 1-й уровень
- На уровне 1 вы обнаружите, что слова, описывающие нежелательную реакцию, точно такие же, как понятия CUI.
> Вот пример:
> - Sentence text: `I was unable to sleep, had blurred vision, and felt sick to my stomach.`
> - ADR: **blurred vision**
> - CUI concept: **blurred vision**
> - CUI: **C0344232** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

- Кроме того, слова определяются как «одинаковые», даже если они представляют собой разные производные (обычно разные суффиксы в английском языке) одного и того же слова. Те же леммы или те же основы примерно.
>Вот еще один пример, который в этом наборе данных считается уровнем 1:
> - Sentence text: `Muscle spasms, muscle twitching, muscle soreness, insomnia, mental confusion, flush, brain zaps.`
> - ADR: **flush**
> - CUI Concept: **flushing**
> - CUI: **C0016382** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

## Уровень 1 — Точное совпадение
- Давайте начнем с извлечения примеров только уровня 1. Все, что нам нужно сделать, это посмотреть, совпадают ли точные слова:
   - Сначала выберем метод предварительной обработки.
   - Тогда мы извлечем АДР

In [ ]:
porter_stemmer = PorterStemmer()
# Это предложение для первого уровня.
# Убедитесь, что вы понимаете, что он делает, что в этом может помочь

def text_preprocessing_1(sentence):
    """Read in a sentence and return a list of tokens
       which are lower case and punctuation free.
       sentence must be a string"""
     # убираем знаки препинания из предложения и заменяем их на ' '
     # (пункт взят из строкового модуля, импортированного выше)
    s = sentence.translate(sentence.maketrans(punct, ' '*len(punct)))
    # простая токенизация
    toks = s.split()
    # нижний регист
    lowered = [w.lower() for w in toks]
    stemmed = [porter_stemmer.stem(w) for w in lowered]
    joined = ' '.join(stemmed)

    return joined
    # return stemmed

In [ ]:
# 1 уровень кода
def check_concept_1(concept, sentence):
    """Проверьте, упоминается ли в предложении концепты CUI.
    Верните TRUE, если концепт присутствует, верните FALSE, если нет """
    # предварительная обработка понятия и предложения
    c_str = text_preprocessing_1(concept)
    s_str = text_preprocessing_1(sentence)
    # print(c_str)
    # print(s_str)

    # Проверяем, является ли текст понятия подпоследовательностью предложения
    return c_str in s_str
    # return all(word in s_str for word in c_str)

In [ ]:
# пример уровеня 1
concept1_1 = 'Blurred vision' # то же CUI как 'Blurring of visual image'
sentence1_1 = 'I was unable to sleep, had blurred vision, and felt sick to my stomach.'

ans1_1 = check_concept_1(concept1_1, sentence1_1) # должно быть True!
print(bool(ans1_1))   # должен печатать True

True


In [ ]:
concept1_2 = 'weight loss'
sentence1_2 = first_sentence

# assert(check_concept_1(concept2_1, sentence2_1), 'Совпадения не найдены')
check_concept_1(concept1_2, sentence1_2)

False

In [ ]:
concept1_3 = 'weight gain'
sentence1_3 = first_sentence

check_concept_1(concept1_3, sentence1_3)

True

In [ ]:
first_sentence

'extreme weight gain, short-term memory loss, hair loss.'

*Если следовать шаблону из лекции, то мы получаем списки токенов (в данном случае лемматизированных слов). Вроде бы в таком случае нужно проверять через all(). Во-всяком случае, я почему-то подумал так, хотя может быть нужно было хитрее проверку сделать.*
*И в таком случае мы в первом предложении "extreme weight gain, short-term memory loss, hair loss." получаем совпадение с ''weight loss''*
*На мой взляд - это неправильно, поэтому изменил первой обработки и стал выводить склеенную строку. И искать только точное совпадение подстроки в строке*

Рассмотрим укороченный датафрем для тестов.

In [ ]:
# df_small = df.iloc[0:5]
# df_small

In [ ]:
# создадим датайфрейм, который будем заполнять.
data = pd.DataFrame(columns=['row_id', 'sentense_id', 'CUI', 'level_1', 'level_2',
                             'level_3', 'level_4', 'level_5'])

In [ ]:
def table_forming(df_s, df_c, txt_prep, check, data, lvl):
    '''
    Функция заполняет датафрем. Исходя из проверки на определённом уровне извлечения концептов
    '''
    for row in range(len(df_s)):
          # проходим по строкам
          # print(df_s.iloc[row][1])
          for cncpt in range(len(df_c)):
              # проходим по концептам
              # print(check(df_c.iloc[cncpt][1], df_s.iloc[sent][1]))
              if check(df_c.iloc[cncpt][1], df_s.iloc[row][1]):
                  # если проходим проверку, то добавляем CUI концепта в датафрейм
                  data = data.append({'row_id': row,
                                      'sentense_id': df_s.iloc[row]['sentense_id'],
                                      'CUI': df_c.iloc[cncpt]['CUI'],
                                      f'level_{lvl}': 1,}, ignore_index=True)

    return data

In [ ]:
df_concepts[df_concepts['CUI'] == 'C0000765']

,CUI,concepts
0,C0000765,excessive body weight gain
1,C0000765,excessive weight gain


In [ ]:
# 'weight decreased' in 'excessive body weight gain'

In [ ]:
# таблица, полученная после применения проверки на первом уровне извлечения
# data_f1 = table_forming(df_small, df_concepts, text_preprocessing_1, check_concept_1, data, 1)
data_f1 = table_forming(df, df_concepts, text_preprocessing_1, check_concept_1, data, 1)

In [ ]:
data_f1.head()

,row_id,sentense_id,CUI,level_1,level_2,level_3,level_4,level_5
0,0,1,C0043094,1,NaN,NaN,NaN,NaN
1,1,2,C0036104,1,NaN,NaN,NaN,NaN
2,2,3,C0036104,1,NaN,NaN,NaN,NaN
3,3,4,C0549622,1,NaN,NaN,NaN,NaN
4,5,6,C0027497,1,NaN,NaN,NaN,NaN


In [ ]:
first_sentence

'extreme weight gain, short-term memory loss, hair loss.'

In [ ]:
df_concepts[df_concepts['CUI'] == 'C0701811']

,CUI,concepts
2,C0701811,poor short-term memory


In [ ]:
df_concepts[df_concepts['CUI'] == 'C0036104']

,CUI,concepts
718,C0036104,salivary secretion
719,C0036104,secretion of saliva
720,C0036104,function


*Вообще получается, что слишком большой разброс, например, в этом концепте. function - вообще слишком большое понятие*

## 2-й уровень

 — Точное совпадение, возможно, не по порядку

 #### Уровень 2
- На уровне 2 вы можете обнаружить, что слова, описывающие нежелательную реакцию, совпадают с понятиями CUI, хотя порядок слов может быть другим.
> Вот пример:
> - Sentence text: `Weight gain, HAIR LOSS, increased depression, fatigue, lethargy.`
> - ADR: **HAIR LOSS**
> - CUI concept: **Loss of hair**
> - CUI: **C0002170** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

In [ ]:
# !pip install thefuzz

In [ ]:
# from thefuzz import fuzz
# fuzz.ratio("poor short-term memory", "extreme weight gain, short-term memory loss, hair loss.")

*Здесь совсем не смог придумать нормальных схем. Удаление стопслов - это хороший способ, но мы найдём много лишних вариантов*

*Т.е. останется проблема нахождения сущности ''weight loss'' в  "extreme weight gain, short-term memory loss, hair loss." Если мы просто оставим список токенов*

In [ ]:
# porter_stemmer = PorterStemmer()

# def text_preprocessing_2(sentence, stemm=True):
#     """Прочитайте предложение и верните список токенов, в которых нет основы, нет знаков препинания и стоп-слов.
#         предложение должно быть строкой"""
#     # Все предоставленное text_preprocessing_1 в одной строке
#     toks = [porter_stemmer.stem(w.lower()) if stemm else w.lower() for w in
#              sentence.translate(sentence.maketrans(punct, ' '*len(punct))).split()]
#     # toks = text_preprocessing_1(sentence) # эквивалентно, если вы не меняли text_preprocessing_1

#     # Получить список стоп-слов из известного источника
#     stopwords = nltk.corpus.stopwords.words('english')

#     no_stopword = [tok for tok in toks if tok.lower() not in stopwords]
#      # Вы можете изменить эту функцию, если считаете, что она вам поможет.
#      # Не стесняйтесь добавлять или удалять шаги!

#     final = no_stopword
#     return final

In [ ]:
# # 2 уровень кода
# def check_concept_2(concept, sentence):
#     """Проверьте, появляется ли в предложении концепция CUI, даже если слова не в правильном порядке.
#     Верните TRUE, если концепция присутствует, верните FALSE, если нет."""
#     # предварительная обработка понятия и предложения
#     c_lst = set(text_preprocessing_2(concept))
#     s_lst = set(text_preprocessing_2(sentence))

#     if c_lst.issubset(s_lst):
#         return True
#     else:
#         return False

#     # # Проверяем, является ли текст понятия подпоследовательностью предложения
#     # return all(word in s_lst for word in c_lst)

*Попробуем очистить от стоп слов и повторить вариант первого шага, когда мы собирали строку.*

In [ ]:
porter_stemmer = PorterStemmer()

def text_preprocessing_2(sentence, stemm=True):
    """Прочитайте предложение и верните список токенов, в которых нет основы, нет знаков препинания и стоп-слов.
        предложение должно быть строкой"""
    # Все предоставленное text_preprocessing_1 в одной строке
    toks = [porter_stemmer.stem(w.lower()) if stemm else w.lower() for w in
             sentence.translate(sentence.maketrans(punct, ' '*len(punct))).split()]
    # toks = text_preprocessing_1(sentence) # эквивалентно, если вы не меняли text_preprocessing_1

    # Получить список стоп-слов из известного источника
    stopwords = nltk.corpus.stopwords.words('english')

    no_stopword = [tok for tok in toks if tok.lower() not in stopwords]

    final = no_stopword
    joined = ' '.join(final)
    return joined

In [ ]:
def check_concept_2(concept, sentence):
    """Проверьте, упоминается ли в предложении концепты CUI.
    Верните TRUE, если концепт присутствует, верните FALSE, если нет """
    # предварительная обработка понятия и предложения
    c_str = text_preprocessing_2(concept)
    s_str = text_preprocessing_2(sentence)
    # print(c_str)
    # print(s_str)

    # Проверяем, является ли текст понятия подпоследовательностью предложения
    return c_str in s_str

In [ ]:
concept3_2 = 'loss of hair'
sentence3_2 = 'extreme weight gain, short-term memory loss, hair loss.'

check_concept_2(concept3_2, sentence3_2)

True

*как видно у этого способа свои минусы: неверно определяется loss of hair.*

In [ ]:
def reverse_word(s):
    words = s.split(' ')
    string = []
    for word in words:
        string.insert(0, word)

    return (" ".join(string))

In [ ]:
# проверим ещё и развёрнутые концепты. Пусть это рабочий вариант только для концептов
# с двумя словами
df_concepts_reversed = df_concepts.copy()
df_concepts_reversed['concepts'] = df_concepts_reversed['concepts'].apply(reverse_word)
# df_concepts_reversed.head()

In [ ]:
%%time
# data_f2 = table_forming(df_small, df_concepts, text_preprocessing_2, check_concept_2, data, 2)
data_f2 = table_forming(df, df_concepts, text_preprocessing_2, check_concept_2, data, 2)

CPU times: user 17min 55s, sys: 52.4 s, total: 18min 47s
Wall time: 18min 59s


In [ ]:
%%time
# data_f3 = table_forming(df_small, df_concepts_reversed, text_preprocessing_2, check_concept_2, data, 2)
data_f3 = table_forming(df, df_concepts_reversed, text_preprocessing_2, check_concept_2, data, 2)

CPU times: user 17min 57s, sys: 53 s, total: 18min 50s
Wall time: 19min


In [ ]:
# df_concepts[df_concepts['CUI'] == 'C0557386']

## 3-ий уровень.

- В отличие от уровней 1 и 2, где необходимо было идентифицировать слова с одним и тем же корнем, но разной формой, на уровне 3 возникает необходимость идентифицировать слова с существенно отличающейся формой, но имеющими «достаточно схожее» значение.
- На уровне 3 вы можете попытаться рассматривать эти пары слов, которые «достаточно похожи», как совпадения слов. Вы можете обнаружить, что это работает.
- Вы также можете не слишком полагаться на сопоставление слов и придумать другие методы для извлечения большего количества ADR из уровня 3. Это открытый метод, и если вам кажется более естественным другой подход, вы можете попробовать его.


- На уровне 3 вы можете обнаружить, что, помимо разницы в порядке слов, только некоторые слова, описывающие нежелательную реакцию, точно такие же, как в понятиях CUI.
> Вот пример:
> - Sentence text: `It's been four days since I discontinued, and I'm still wiped out and dizzy with flu-like symptoms.`
> - ADR: **flu-like symptoms**
> - CUI concept: **Influenza-like symptoms**
> - CUI: **C0392171** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

### Семантическая близость

*Здесь, насколько я понимаю, можно попробовать найти семантическую близость*

In [ ]:
word_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
def text_preprocessing_3(sentence, stemm=True):
    """Прочитайте предложение и верните список токенов, в которых нет основы, нет знаков препинания и стоп-слов."""
    toks = [porter_stemmer.stem(w.lower()) if stemm else w.lower() for w in
                sentence.translate(sentence.maketrans(punct, ' '*len(punct))).split()]
    stopwords = nltk.corpus.stopwords.words('english')
    no_stopword = [tok for tok in toks if tok.strip() not in stopwords]

    final = no_stopword
    return final

def check_concept_3(concept, sentence, threshold=0.7):
    """Проверьте, появляется ли в предложении концепция CUI с семантической близостью,
    превышающей порог. Верните TRUE, если концепция присутствует, верните FALSE, если нет."""
    # предварительная обработка понятия и предложения
    c_lst = text_preprocessing_3(concept)
    s_lst = text_preprocessing_3(sentence)

    # Измерение семантической близости между понятием и предложением
    similarity_score = word_vectors.n_similarity(c_lst, s_lst)

    # Проверка, превышает ли близость порог
    return similarity_score > threshold


In [ ]:
# 3 уровень кода
def check_concept_3(concept, sentence):
    """Проверьте, появляется ли в предложении концепция CUI, даже если слова имеют разные корни, но схожие значения.
    Верните TRUE, если концепция присутствует, верните FALSE, если нет."""
    # предварительная обработка понятия и предложения
    c_lst = text_preprocessing_3(concept)
    s_lst = text_preprocessing_3(sentence)

    # Проверяем, есть ли слова схожего значения в предложении
    return any(word in s_lst for word in c_lst)


### Синонимы

*Но попробуем сначала синонимы из WordNet*

In [ ]:
# def text_preprocessing_3(sentence, stemm=True):
#     """Read a sentence and return a list of tokens with no stemming, no punctuation and no stop words."""
#     # Tokenize the sentence and convert to lowercase
#     toks = [porter_stemmer.stem(w.lower()) if stemm else w.lower() for w in sentence.split()]
#     # Remove punctuation
#     toks = [''.join(c for c in w if c not in punct) for w in toks]
#     # Remove stop words
#     stopwords = nltk.corpus.stopwords.words('english')
#     no_stopword = [tok for tok in toks if tok.strip() not in stopwords]
#     # Remove leading and trailing spaces and extra spaces
#     final = [tok.strip() for tok in no_stopword if tok.strip()]
#     return final

In [ ]:
def text_preprocessing_3(sentence, stemm=True):
    """Прочитайте предложение и верните список токенов, в которых нет основы, нет знаков препинания и стоп-слов."""
    toks = [porter_stemmer.stem(w.lower()) if stemm else w.lower() for w in
                sentence.translate(sentence.maketrans(punct, ' '*len(punct))).split()]
    stopwords = nltk.corpus.stopwords.words('english')
    no_stopword = [tok for tok in toks if tok.strip() not in stopwords]

    final = no_stopword
    return final

In [ ]:
def check_concept_3(concept, sentence, top_n=1):

    c_lst = text_preprocessing_3(concept)
    s_lst = text_preprocessing_3(sentence)

    # Find synonyms of words in the concept using WordNet
    synonyms = list()
    for word in c_lst:
        for syn in wn.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())

    # Check if any of the synonyms appear in the sentence
    for word in s_lst:
        if word in synonyms[:top_n]:
            return True

    return False

In [ ]:
concept3 = 'Severe vertigo,Severe vertigo'
sentence3 = 'Emergency Room visit with tachycardia and violent vertigo.'

print(check_concept_3(concept3, sentence3)) # Output: True

False


In [ ]:
concept3 = 'excessive body weight gain'
sentence3 = first_sentence

print(check_concept_3(concept3, sentence3, top_n=25))

True


In [ ]:
concept3 = 'weight loss'
sentence3 = first_sentence

print(check_concept_3(concept3, sentence3))

True


*Пословное сравнение, даёт свои минусы. Выше пример неверного обнаружения. Впрочем, это не совсем из-за синонимов, а из-за поиска синонимов к каждому слову предложения.*
*Ещё из минусов подбор количества топ синонимов, которое используем. В конечном итоге обнаруживалось слишком много концептов.*

In [ ]:
%%time
# data_f4 = table_forming(df_small, df_concepts_reversed, text_preprocessing_3, check_concept_3, data, 3)
data_f4 = table_forming(df, df_concepts_reversed, text_preprocessing_3, check_concept_3, data, 3)

CPU times: user 27min 36s, sys: 54.2 s, total: 28min 30s
Wall time: 28min 36s


## 4-ый уровень


- На уровне 4 вы обнаружите, что ни одно из слов, описывающих нежелательную реакцию, не совпадает со словами в концепциях CUI, но при этом имеет то же значение.
> Вот пример:
> - Sentence text: `Also I had a severe inablility to concentrate which made it impossible to do my job,I am a Registered Nurse.`
> - ADR: **impossible to do my job**
> - CUI concept: **Restricted work performance**
> - CUI: **C0557386** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

In [ ]:
# !git clone https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr
# model = Word2Vec.load('w2v_oa_cr_100d.bin')

In [ ]:
# # Загрузить предобученную модель Word2Vec
# model = gensim.models.KeyedVectors.load_word2vec_format('path/to/word2vec.bin', binary=True)

# def text_preprocessing_4(sentence, stemm=True):
#     """Прочитайте предложение и верните список токенов, в которых нет основы, нет знаков препинания и стоп-слов."""
#     toks = [porter_stemmer.stem(w.lower()) if stemm else w.lower() for w in
#                 sentence.translate(sentence.maketrans(punct, ' '*len(punct))).split()]
#     stopwords = nltk.corpus.stopwords.words('english')
#     no_stopword = [tok for tok in toks if tok.strip() not in stopwords]

#     # Преобразовать слова в векторы
#     vecs = [model[tok] for tok in no_stopword if tok in model]

#     return vecs

# def check_concept_4(concept, sentence):
#     """Проверьте, упоминается ли в предложении концепция CUI. Возвращайте TRUE, если концепция существует, и возвращайте FALSE, если нет."""
#     c_vec = text_preprocessing_4(concept)
#     s_vec = text_preprocessing_4(sentence)

#     # Найти среднее значение векторов слов в концепции и предложении
#     c_mean = sum(c_vec) / len(c_vec) if c_vec else 0
#     s_mean = sum(s_vec) / len(s_vec) if s_vec else 0

#     # Вычислить косинусное сходство между средними векторами
#     sim = cosine_similarity([c_mean], [s_mean])

#     # Вернуть TRUE, если косинусное сходство превышает пороговое значение
#     return sim > 0.5

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
# model

In [ ]:
# # Sample text
# text = "This is a sample biomedical text."

# # Tokenize the text
# inputs = tokenizer(text, return_tensors="pt")

# # Pass the tokenized input to the model and get the last hidden states
# with torch.no_grad():
#     outputs = model(**inputs)
#     last_hidden_states = outputs.last_hidden_state

# # 'last_hidden_states' is a tensor of shape (batch_size, sequence_length, hidden_size)
# # The CLS token's embedding is stored at the first position of the sequence_length dimension

# # Extract the CLS token's embedding as the sentence embedding
# sentence_embedding = last_hidden_states[0, 0]

In [ ]:
# len(sentence_embedding)
# sentence_embedding = torch.mean(sentence_embedding, dim=0)

In [ ]:
# Sample text
text = "This is a sample biomedical text."

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt")

# Pass the tokenized input to the model and get all hidden states
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)
    encoded_layers = outputs.hidden_states

# Print the shape of the encoded_layers tensor
print()
print("Shape of encoded_layers:", tuple(encoded_layers[0].shape))


Shape of encoded_layers: (1, 10, 768)


In [ ]:
len(encoded_layers)

13

In [ ]:
def text_preprocessing_4(sentence):
    """Прочитайте предложение и верните список токенов, в которых нет основы, нет знаков препинания и стоп-слов."""
    inputs = tokenizer(sentence, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        encoded_layers = outputs.hidden_states

    token_vecs = encoded_layers[12][0]
    # Найти среднее значение векторов слов в предложении
    sentence_embedding = torch.mean(token_vecs, dim=0)
    # print("Our final sentence embedding vector of shape:", sentence_embedding.size())

    return sentence_embedding

def check_concept_4(concept, sentence, threshold=0.85):
    c_vec = text_preprocessing_4(concept)
    s_vec = text_preprocessing_4(sentence)

    # Вычисkztv косинусное сходство между средними векторами
    sim = cosine_similarity([c_vec], [s_vec])

    # Вернуть TRUE, если косинусное сходство превышает пороговое значение
    if sim > threshold:
        return True
    else:
        return False

In [ ]:
concept4 = 'Restricted work performance'
sentence4 = 'Also I had a severe inablility to concentrate which made it impossible to do my job,I am a Registered Nurse.'

check_concept_4(concept4, sentence4, threshold=0.75) # Output: True

True

In [ ]:
concept4 = 'weight gain'
sentence4 = 'Also I had a severe inablility to concentrate which made it impossible to do my job,I am a Registered Nurse.'

check_concept_4(concept4, sentence4) # Output: False

False

In [ ]:
%%time
# data_f5 = table_forming(df_small, df_concepts_reversed, text_preprocessing_4, check_concept_4, data, 4)
# data_f5 = table_forming(df, df_concepts, text_preprocessing_4, check_concept_4, data, 4)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


In [ ]:
# data_f5.to_csv('data_f5.csv', index=False)

*Порог близости был взят наугад.*


#### Уровень 5 &nbsp; &nbsp; (От вас не ожидается, что вы их решите)
- На уровне 5 вы можете обнаружить, что слова, описывающие побочную реакцию, не совпадают с понятиями CUI и имеют разные значения. ADR может предлагать концепцию CUI посредством следствия или посредством какого-либо другого лингвистического или контекстуального механизма. Если вы сумеете их идентифицировать, это будет очень впечатляюще!
> Вот пример:
> - Sentence text: `20 pounds total in 6 months Appetite increase .`
> - ADR: **20 pounds total**
> - CUI concept: **Body Weight Changes**
> - CUI: **C0005911** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

*Не выполнил. В принципе я и в прошлых уровнях не сильно уверен, что там правильное разделение сделано.*

# Объединение полученных данных и удаление дублей

In [ ]:
data_f = pd.concat([data_f1, data_f2, data_f3, data_f4,
                    # data_f5
                    ])
data_f.drop_duplicates(inplace=True)
data_f = data_f.fillna(0)
data_f = data_f.sort_values(by='sentense_id')
data_f[['sentense_id', 'level_1', 'level_2', 'level_3', 'level_4',
        'level_5']] = data_f[['sentense_id', 'level_1', 'level_2',  'level_3',
                              'level_4',	'level_5']].astype(int)
data_f.head()

,row_id,sentense_id,CUI,level_1,level_2,level_3,level_4,level_5
14685,2109,0,C0018784,0,0,1,0,0
14683,2109,0,C0011057,0,0,1,0,0
2862,2109,0,C0042571,1,0,0,0,0
2861,2109,0,C0012833,1,0,0,0,0
14669,2109,0,C1971624,0,0,1,0,0


In [ ]:
data_f.to_csv('data_f.csv', index=False)

In [ ]:
# data_f['level_4'].unique()

array([0, 1])

In [ ]:
# df_concepts[df_concepts['CUI'] == 'C2364111']

,CUI,concepts
217,C2364111,ageusia
218,C2364111,loss of taste


In [ ]:
# df_1 = data_f[data_f['sentense_id']== 1]
# df_1

In [ ]:
df_sub = data_f.copy()
duplicates = df_sub.duplicated(subset=['sentense_id', 'CUI'], keep='first')

df_sub['flag'] = duplicates.astype(int)

df_sub = df_sub.query('flag != 1')
df_sub.head()

,row_id,sentense_id,CUI,level_1,level_2,level_3,level_4,level_5,flag
14685,2109,0,C0018784,0,0,1,0,0,0
14683,2109,0,C0011057,0,0,1,0,0,0
2862,2109,0,C0042571,1,0,0,0,0,0
2861,2109,0,C0012833,1,0,0,0,0,0
14669,2109,0,C1971624,0,0,1,0,0,0


In [ ]:
df_sub = df_sub[['sentense_id',	'CUI',	'level_1',	'level_2',	'level_3',	'level_4',	'level_5']]
df_sub.to_csv('Berezutskiy_3.csv', index=False, header=False)

In [ ]:
# df_effects = pd.merge(data_f, df_concepts, on='CUI', how='left')

In [ ]:
# df_t = df_effects[df_effects['sentense_id'] == '1']
# df_t

In [ ]:
# df_t = df_t.drop_duplicates()

# О работе



1) **Не успел досчитать. В данном методе успел посмотреть только на части датасета.**

На мой взгляд, самым продуктивным способом было сравнение семантической близости на основе предобученной модели на биомедицинских данных - самый удачный способ из этих всех. Минусом является порог, который нужно подбирать. А для подбора тогда лучше бы иметь предобученные данные. (Возможно модель, обученная на клинических, дала бы лучшие результаты)

2) Самым нормально организованным способом был самый первый уровень. На мой взгляд, он вроде бы выполняет то, что должен: поиск именно точного соответствия. Реализован был поиск подстроки в строке.

3) Второй и третий уровень реализованы не очень хорошо.

На втором уровне: мы выбросили не только знаки препинания, но и стоп-слова. Далее тоже поиск подстроки. Помимо этого, ещё один проход для с перевёрнутыми словами в концепте.

Во втором можно было бы сравнивать просто списки слов в концепте и в предложении, пусть были бы ложные срабатывания, но возможно их было бы не столь много.


4) Третий уровень, где рассматривал слова-синонимы из WordNet, в принципе, если рассматривать каждое отдельное предложение и выбирать подходящий топ, то этот способ работает. Проблема в том, что в такой реализации неизвестно как подобрать какое количество синонимов можно использовать. И получается слишком много ложных срабатываний

5) Может быть стоило составить эмбединги слов word2vec или чем-то подобным, чтобы получить эмбединги с контекстом. Но вообще мне кажется датасет мал, чтобы получить что-то стоящее. Поэтому предопложил, что это совсем бесполезное занятие.

6) Возможно стоит собрать из разных уровней подобные ответы, но создать вес для каждого уровня и тем самым удалить какие-нибудь ложные варианты.

7) Сами концепты достаточно пересекаются. weight gain входит в excessive body weight gain,	excessive weight gain. Это тоже сильно влияет на качество мэтчинга.